# Notebook to generate psychometrics plot given sessions & conditions

In [1]:
import datajoint as dj
import numpy as np
import pylab as plt
import pandas as pd
import json
import datetime
pd.set_option('display.max_rows', 50)

### Datajoint configuration

### Connection to DB

In [2]:
conn = dj.conn()
action = dj.create_virtual_module('action', 'u19_test_action')
subject = dj.create_virtual_module('subject', 'u19_test_subject')
lab = dj.create_virtual_module('lab', 'u19_test_lab')

[2025-06-21 12:29:56,103][INFO]: Connecting alvaros@datajoint00.pni.princeton.edu:3306
[2025-06-21 12:29:57,761][INFO]: Connected alvaros@datajoint00.pni.princeton.edu:3306


In [3]:
def index_act(act_list):
    occurrence_dict = {}
    for index, item in enumerate(act_list):
        occurrence_dict.setdefault(item, []).append(index)
    return occurrence_dict

In [4]:
all_subject_status = pd.DataFrame((action.SubjectStatus * subject.Subject.proj('user_id') * lab.User.proj('tech_responsibility')).fetch(as_dict=True))
all_subject_status = all_subject_status.sort_values(by='effective_date', ascending=False)
all_subject_status = all_subject_status.drop_duplicates(subset='subject_fullname', keep='first')

all_subject_status = all_subject_status.sort_values(by='effective_date', ascending=False)

all_subject_status = all_subject_status.reset_index(drop=True)

all_subject_status['schedule_list'] = all_subject_status['schedule'].apply(lambda x: x.split('/'))
all_subject_status['schedule_set'] = all_subject_status['schedule_list'].apply(lambda x: set(x))
all_subject_status['num_act'] = all_subject_status['schedule_set'].apply(lambda x: len(x))
all_subject_status['index_act'] = all_subject_status['schedule_list'].apply(lambda x: index_act(x))

## Activities when one person does everything

In [5]:
subject_status_dict_owner = {
    'Dead': ['Nothing'],
    'AdLibWater': ['Nothing'],
    'Missing': ['Nothing'],
    'WaterRestrictionOnly': ['Watering', 'Weighing'],
    'InExperiments': ['Watering', 'Weighing', 'Training']
}
subject_status_dict_technician = {
    'Dead': ['Nothing'],
    'AdLibWater': ['Nothing'],
    'Missing': ['Nothing'],
    'WaterRestrictionOnly': ['Watering', 'Weighing', 'Transport'],
    'InExperiments': ['Watering', 'Weighing', 'Training', 'Transport']
}

act_to_status_dict = {
    'Water': 'WaterRestrictionOnly',
    'Weigh': 'WaterRestrictionOnly',
    'Train': 'InExperiments',
    'OnlyTrain': 'InExperiments',
    'Nothing': 'Dead',
    'Transport': 'Dead'

}

## Activities when activities are shared (assuming weekends techs do everything)

In [6]:
subject_status_dict_owner_shared = {
    'Dead': ['Nothing'],
    'AdLibWater': ['Nothing'],
    'Missing': ['Nothing'],
    'WaterRestrictionOnly': ['Watering', 'Weighing'],
    'InExperiments': ['Watering', 'Weighing', 'Training']
}
subject_status_dict_technician_shared = {
    'Dead': ['Nothing'],
    'AdLibWater': ['Nothing'],
    'Missing': ['Nothing'],
    'WaterRestrictionOnly': ['Watering', 'Weighing', 'Transport'],
    'InExperiments': ['Watering', 'Weighing', 'Training', 'Transport']
}

act_to_status_dict_shared = {
    'Water': 'WaterRestrictionOnly',
    'Weigh': 'WaterRestrictionOnly',
    'Train': 'InExperiments',
    'OnlyTrain': 'InExperiments',
    'Nothing': 'Dead',
    'Transport': ['Transport']

}

In [7]:
base_weekly_dict = {
  "assignment_style": "Specify for the whole week",
  "weekly": ['Nothing']
}

base_weekdays_weekends_dict = {
  "assignment_style": "Specify by weekday/weekend",
  "weekdays": ['Nothing'],
  "weekends": ['Nothing']
}


base_days_dict = {
"assignment_style":"Specify each day",
"Monday": ['Nothing'],
"Tuesday": ['Nothing'],
"Wednesday": ['Nothing'],
"Thursday": ['Nothing'],
"Friday": ['Nothing'],
"Saturday": ['Nothing'],
"Sunday": ['Nothing'],
}


In [8]:
def translate_schedule(df_row):

    user_id = df_row['user_id']
    schedule_list = df_row['schedule_list']
    tech_responsibility = df_row['tech_responsibility']
    subject_status = df_row['subject_status']
    num_act = df_row['num_act']
    index_act = df_row['index_act']


    tech_act_dict = base_weekly_dict.copy()
    tech_act_dict['weekly'] = ['Nothing']
    owner_act_dict = base_weekly_dict.copy()
    owner_act_dict['weekly'] = ['Nothing']

    if tech_responsibility == 'yes':
        if subject_status in ('Dead', 'AdLibWater', 'Missing'):
            tech_act_dict = base_weekly_dict.copy()
            tech_act_dict['weekly'] = subject_status_dict_technician[subject_status]
        elif num_act == 1:
            tech_act_dict = base_weekly_dict.copy()
            tech_act_dict['weekly'] = subject_status_dict_technician[subject_status]
        elif num_act == 2 and list(index_act.values())[0] == [1,2,3,4,5] or list(index_act.values())[0] == [0,6]:

            tech_act_dict = base_weekdays_weekends_dict.copy()
            weekdays_act = schedule_list[1]
            weekdends_act = schedule_list[6]
            if weekdays_act == 'Transport' or weekdends_act == 'Transport':
                owner_act_dict = base_weekdays_weekends_dict
            if weekdays_act == 'Transport':
                tech_act_dict['weekdays'] = ['Transport']
                owner_act_dict['weekdays'] = subject_status_dict_owner[subject_status]
            else:
                weekdays_status = act_to_status_dict[weekdays_act]
                tech_act_dict['weekdays'] = subject_status_dict_technician[weekdays_status]
            if weekdends_act == 'Transport':
                tech_act_dict['weekends'] = ['Transport']
                owner_act_dict['weekends'] = subject_status_dict_owner[subject_status]
            else:
                weekdends_status = act_to_status_dict[weekdends_act]
                tech_act_dict['weekends'] = subject_status_dict_technician[weekdends_status]
        else:
            tech_act_dict = base_days_dict.copy()

            

            tech_act_dict['Monday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[0]]]
            tech_act_dict['Tuesday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[1]]]
            tech_act_dict['Wednesday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[2]]]
            tech_act_dict['Thursday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[3]]]
            tech_act_dict['Friday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[4]]]
            tech_act_dict['Saturday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[5]]]
            tech_act_dict['Sunday'] = subject_status_dict_technician[act_to_status_dict[schedule_list[6]]]
    
    # Tech_responsibility = no
    else:
        if subject_status not in ('Dead', 'AdLibWater', 'Missing'):
            tech_act_dict['weekly'] = ['Transport']
            if num_act == 1:
                owner_act_dict['weekly'] = subject_status_dict_owner[subject_status]
            elif num_act == 2 and list(index_act.values())[0] == [1,2,3,4,5] or list(index_act.values())[0] == [0,6]:
                owner_act_dict = base_weekdays_weekends_dict.copy()
                weekdays_act = schedule_list[1]
                weekdends_act = schedule_list[6]
                owner_act_dict['weekdays'] = subject_status_dict_owner[subject_status]
                owner_act_dict['weekends'] = subject_status_dict_owner[subject_status]
            else:
                owner_act_dict = base_days_dict.copy()
                schedule_list = [act_to_status_dict[x] if x!='Transport' else x for x in schedule_list]
                schedule_list = [subject_status if x=='Transport' else x for x in schedule_list]

                owner_act_dict['Monday'] = subject_status_dict_owner[schedule_list[0]]
                owner_act_dict['Tuesday'] = subject_status_dict_owner[schedule_list[1]]
                owner_act_dict['Wednesday'] = subject_status_dict_owner[schedule_list[2]]
                owner_act_dict['Thursday'] = subject_status_dict_owner[schedule_list[3]]
                owner_act_dict['Friday'] = subject_status_dict_owner[schedule_list[4]]
                owner_act_dict['Saturday'] = subject_status_dict_owner[schedule_list[5]]
                owner_act_dict['Sunday'] = subject_status_dict_owner[schedule_list[6]]
    
                
    return pd.Series({'technician_duties': json.dumps(tech_act_dict), 'owner_duties': json.dumps(owner_act_dict)}) 


            



In [9]:
all_subject_status[['technician_duties', 'owner_duties']] = all_subject_status.apply(translate_schedule, axis=1)

In [10]:
all_subject_status

,subject_fullname,effective_date,user_id,subject_status,water_per_day,schedule,tech_responsibility,schedule_list,schedule_set,num_act,index_act,technician_duties,owner_duties
0,testuser_T36,2025-06-16,testuser,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,no,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1,testuser_monday_tria,2025-06-10,testuser,WaterRestrictionOnly,1.0,Train/Train/Train/Train/Train/Train/Train,no,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
2,testuser_captian_america,2025-06-10,testuser,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,no,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
3,jk8386_jk67,2025-06-09,jk8386,AdLibWater,1.0,Train/Train/Train/Train/Train/Train/Train,yes,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
4,jk8386_jk66,2025-06-09,jk8386,InExperiments,1.0,Train/Train/Train/Train/Train/Train/Train,yes,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,hnieh_E36,2017-09-10,hnieh,Dead,1.5,Water/Nothing/OnlyTrain/OnlyTrain/OnlyTrain/On...,yes,"[Water, Nothing, OnlyTrain, OnlyTrain, OnlyTra...","{OnlyTrain, Water, Train, Nothing}",4,"{'Water': [0], 'Nothing': [1], 'OnlyTrain': [2...","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1167,hnieh_E18,2017-08-16,hnieh,Dead,1.5,Water/Nothing/OnlyTrain/OnlyTrain/OnlyTrain/On...,yes,"[Water, Nothing, OnlyTrain, OnlyTrain, OnlyTra...","{OnlyTrain, Water, Train, Nothing}",4,"{'Water': [0], 'Nothing': [1], 'OnlyTrain': [2...","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1168,koay_K55,2017-08-12,koay,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,yes,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1169,lpinto_vg1,2017-08-02,lpinto,Dead,1.0,Train/Train/Train/Train/Train/Train/Train,yes,"[Train, Train, Train, Train, Train, Train, Train]",{Train},1,"{'Train': [0, 1, 2, 3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."


In [11]:
all_subject_status['technician_duties'].unique()

array(['{"assignment_style": "Specify for the whole week", "weekly": ["Nothing"]}',
       '{"assignment_style": "Specify for the whole week", "weekly": ["Transport"]}',
       '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing", "Training", "Transport"]}',
       '{"assignment_style": "Specify by weekday/weekend", "weekdays": ["Transport"], "weekends": ["Watering", "Weighing", "Transport"]}',
       '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing", "Transport"]}'],
      dtype=object)

In [12]:
all_subject_status['owner_duties'].unique()

array(['{"assignment_style": "Specify for the whole week", "weekly": ["Nothing"]}',
       '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing"]}',
       '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing", "Training"]}',
       '{"assignment_style": "Specify each day", "Monday": ["Nothing"], "Tuesday": ["Nothing"], "Wednesday": ["Nothing"], "Thursday": ["Watering", "Weighing", "Training"], "Friday": ["Watering", "Weighing", "Training"], "Saturday": ["Watering", "Weighing", "Training"], "Sunday": ["Watering", "Weighing", "Training"]}',
       '{"assignment_style": "Specify by weekday/weekend", "weekdays": ["Watering", "Weighing", "Training"], "weekends": ["Nothing"]}',
       '{"assignment_style": "Specify by weekday/weekend", "weekdays": ["Watering", "Weighing"], "weekends": ["Nothing"]}',
       '{"assignment_style": "Specify by weekday/weekend", "weekdays": ["Watering", "Weighing", "Training"], "weekends": ["Waterin

In [13]:
all_subject_status.loc[all_subject_status['owner_duties'].str.contains('{"assignment_style": "Specify each day",'),:]

,subject_fullname,effective_date,user_id,subject_status,water_per_day,schedule,tech_responsibility,schedule_list,schedule_set,num_act,index_act,technician_duties,owner_duties
10,testuser_asdfsd,2025-05-27,testuser,InExperiments,1.0,Nothing/Nothing/Nothing/Train/Train/Train/Train,no,"[Nothing, Nothing, Nothing, Train, Train, Trai...","{Train, Nothing}",2,"{'Nothing': [0, 1, 2], 'Train': [3, 4, 5, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify each day"", ""Mond..."
98,testuser_meow_meow_2,2025-04-15,testuser,WaterRestrictionOnly,1.0,Transport/Weigh/Nothing/Transport/Nothing/Noth...,no,"[Transport, Weigh, Nothing, Transport, Nothing...","{Transport, Weigh, Nothing}",3,"{'Transport': [0, 3], 'Weigh': [1], 'Nothing':...","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify each day"", ""Mond..."
533,testuser_christian_1,2024-02-23,testuser,WaterRestrictionOnly,1.0,Water/Water/Weigh/Water/Weigh/Water/Weigh,no,"[Water, Water, Weigh, Water, Weigh, Water, Weigh]","{Weigh, Water}",2,"{'Water': [0, 1, 3, 5], 'Weigh': [2, 4, 6]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify each day"", ""Mond..."
1091,testuser_T21,2020-03-23,testuser,WaterRestrictionOnly,10.0,Train/Train/Transport/Train/Train/Train/Train,no,"[Train, Train, Transport, Train, Train, Train,...","{Transport, Train}",2,"{'Train': [0, 1, 3, 4, 5, 6], 'Transport': [2]}","{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify each day"", ""Mond..."


In [14]:
all_subject_status_final = all_subject_status.loc[:, ['subject_fullname', 'subject_status', 'water_per_day', 'technician_duties', 'owner_duties']]
all_subject_status_final

,subject_fullname,subject_status,water_per_day,technician_duties,owner_duties
0,testuser_T36,Dead,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1,testuser_monday_tria,WaterRestrictionOnly,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
2,testuser_captian_america,InExperiments,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
3,jk8386_jk67,AdLibWater,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
4,jk8386_jk66,InExperiments,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
...,...,...,...,...,...
1166,hnieh_E36,Dead,1.5,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1167,hnieh_E18,Dead,1.5,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1168,koay_K55,Dead,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."
1169,lpinto_vg1,Dead,1.0,"{""assignment_style"": ""Specify for the whole we...","{""assignment_style"": ""Specify for the whole we..."


In [15]:
all_subject_status_final.to_dict('records')

[{'subject_fullname': 'testuser_T36',
  'subject_status': 'Dead',
  'water_per_day': 1.0,
  'technician_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Nothing"]}',
  'owner_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Nothing"]}'},
 {'subject_fullname': 'testuser_monday_tria',
  'subject_status': 'WaterRestrictionOnly',
  'water_per_day': 1.0,
  'technician_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Transport"]}',
  'owner_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing"]}'},
 {'subject_fullname': 'testuser_captian_america',
  'subject_status': 'InExperiments',
  'water_per_day': 1.0,
  'technician_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Transport"]}',
  'owner_duties': '{"assignment_style": "Specify for the whole week", "weekly": ["Watering", "Weighing", "Training"]}'},
 {'subject_fullname': 'jk8386_jk67',
  'subject_status'

In [16]:
action.Responsibilities.insert(all_subject_status_final.to_dict('records'))

In [17]:
all_subject_status_final.loc[288,:]

subject_fullname                                     testuser_test1000
subject_status                                                 Missing
water_per_day                                                      5.0
technician_duties    {"assignment_style": "Specify for the whole we...
owner_duties         {"assignment_style": "Specify for the whole we...
Name: 288, dtype: object